In [1]:
import akshare as ak
import xgboost as xgb
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV
from time import sleep
import pandas as pd
stock_board_industry_index_ths_df = ak.stock_board_industry_index_ths(symbol="半导体及元件")
# stock_board_industry_index_ths_df = stock_board_industry_index_ths_df.drop([u'日期'],1)
stock_board_industry_index_ths_df

,日期,开盘价,最高价,最低价,收盘价,成交量,成交额
0,2007-08-01,1000.809,1000.809,941.150,941.150,163849330,1747732000.000
1,2007-08-02,943.120,971.811,943.120,971.811,139689670,1546399600.000
2,2007-08-03,977.778,991.772,977.778,991.772,156095660,1897575100.000
3,2007-08-06,998.443,1010.531,998.443,1010.531,171005760,2087825100.000
4,2007-08-07,1012.746,1012.746,1011.061,1011.061,183370260,2201639300.000
...,...,...,...,...,...,...,...
3446,2021-09-29,7942.405,7963.309,7781.852,7801.162,1039800870,41807931000.000
3447,2021-09-30,7831.047,8002.170,7831.047,7983.884,847932000,37553976000.000
3448,2021-10-08,8087.798,8176.094,8001.624,8022.954,964242030,45041209000.000
3449,2021-10-11,8045.188,8089.089,7978.642,7999.060,911093660,42630062000.000


In [62]:
stock_board_industry_index_ths_df = pd.to_numeric(stock_board_industry_index_ths_df.columns, errors='coerce')
stock_board_industry_index_ths_df

Float64Index([nan, nan, nan, nan, nan, nan, nan], dtype='float64')

In [63]:
print(ak.stock_board_industry_name_ths())


      name                                               url
0   半导体及元件  http://q.10jqka.com.cn/thshy/detail/code/881121/
1     白色家电  http://q.10jqka.com.cn/thshy/detail/code/881131/
2    保险及其他  http://q.10jqka.com.cn/thshy/detail/code/881156/
3     包装印刷  http://q.10jqka.com.cn/thshy/detail/code/881138/
4       传媒  http://q.10jqka.com.cn/thshy/detail/code/881164/
..     ...                                               ...
71      证券  http://q.10jqka.com.cn/thshy/detail/code/881157/
72      中药  http://q.10jqka.com.cn/thshy/detail/code/881141/
73    专用设备  http://q.10jqka.com.cn/thshy/detail/code/881118/
74      造纸  http://q.10jqka.com.cn/thshy/detail/code/881137/
75  种植业与林业  http://q.10jqka.com.cn/thshy/detail/code/881101/

[76 rows x 2 columns]


In [64]:
stock_ind_list = ak.stock_board_industry_name_ths()['name'].to_list()
print(stock_ind_list)
print(len(stock_ind_list))


['半导体及元件', '白色家电', '保险及其他', '包装印刷', '传媒', '厨卫电器', '电力', '电力设备', '电子化学品', '房地产服务', '房地产开发', '非金属材料', '非汽车交运', '服装家纺', '纺织制造', '国防军工', '贵金属', '港口航运', '公路铁路运输', '钢铁', '光学光电子', '工业金属', '环保', '化工合成材料', '互联网电商', '黑色家电', '化学原料', '化学制品', '化学制药', '机场航运', '酒店及餐饮', '景点及旅游', '计算机设备', '计算机应用', '金属新材料', '教育', '家用轻工', '建筑材料', '建筑装饰', '零售', '美容护理', '煤炭开采加工', '贸易', '农产品加工', '农业服务', '汽车服务', '汽车零部件', '汽车整车', '其他电子', '其他社会服务', '燃气', '食品加工制造', '生物制品', '石油加工贸易', '通信服务', '通信设备', '通用设备', '物流', '消费电子', '小家电', '小金属', '医疗服务', '医疗器械', '饮料制造', '油气开采及服务', '仪器仪表', '银行', '医药商业', '养殖业', '自动化设备', '综合', '证券', '中药', '专用设备', '造纸', '种植业与林业']
76


In [65]:
#Supress default INFO logging

import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [66]:
industry_result = []
for s in stock_ind_list:
    try:
        industry = ak.stock_board_industry_index_ths(symbol=s)
    except:
        sleep(30)
        industry = ak.stock_board_industry_index_ths(symbol=s)
    industry.rename(columns={u'日期':'date', u'开盘价': 'open', u'最高价':'high',u'最低价':'low', u'收盘价':'close', u'成交量':'volume', u'成交额':'amount'}, inplace=True)
    industry['close'] = pd.to_numeric(industry['close'], errors='coerce')
    industry['open'] = pd.to_numeric(industry['open'], errors='coerce')
    industry['high'] = pd.to_numeric(industry['high'], errors='coerce')
    industry['low'] = pd.to_numeric(industry['low'], errors='coerce')
    industry['volume'] = pd.to_numeric(industry['volume'], errors='coerce')
    industry['amount'] = pd.to_numeric(industry['amount'], errors='coerce')
    industry['EMA_9'] = industry['close'].ewm(9).mean().shift()
    industry['SMA_5'] = industry['close'].rolling(5).mean().shift()
    industry['SMA_10'] = industry['close'].rolling(10).mean().shift()
    industry['SMA_12'] = industry['close'].rolling(12).mean().shift()
    industry['SMA_15'] = industry['close'].rolling(15).mean().shift()
    industry['SMA_20'] = industry['close'].rolling(20).mean().shift()
    industry['SMA_30'] = industry['close'].rolling(30).mean().shift()
    industry['volume_t1'] = industry['volume'].shift(1)
    industry['volume_t2'] = industry['volume'].shift(2)
    industry['volume_t3'] = industry['volume'].shift(3)
    industry['volume_t4'] = industry['volume'].shift(4)
    industry['volume_t5'] = industry['volume'].shift(5)
    industry['volume_t6'] = industry['volume'].shift(6)
    industry['volume_t7'] = industry['volume'].shift(7)
    industry['volume_t8'] = industry['volume'].shift(8)
    industry['volume_t9'] = industry['volume'].shift(9)
    industry['volume_t10'] = industry['volume'].shift(10)
    industry['close_t1'] = industry['close'].shift(1)
    industry['close_t2'] = industry['close'].shift(2)
    industry['close_t3'] = industry['close'].shift(3)
    industry['close_t4'] = industry['close'].shift(4)
    industry['close_t5'] = industry['close'].shift(5)
    industry['close_t6'] = industry['close'].shift(6)
    industry['close_t7'] = industry['close'].shift(7)
    industry['close_t8'] = industry['close'].shift(8)
    industry['close_t9'] = industry['close'].shift(9)
    industry['close_t10'] = industry['close'].shift(10)
    industry['high_t1'] = industry['high'].shift(1)
    industry['high_t2'] = industry['high'].shift(2)
    industry['high_t3'] = industry['high'].shift(3)
    industry['high_t4'] = industry['high'].shift(4)
    industry['high_t5'] = industry['high'].shift(5)
    industry['high_t6'] = industry['high'].shift(6)
    industry['high_t7'] = industry['high'].shift(7)
    industry['high_t8'] = industry['high'].shift(8)
    industry['high_t9'] = industry['high'].shift(9)
    industry['high_t10'] = industry['high'].shift(10)
    industry['low_t1'] = industry['low'].shift(1)
    industry['low_t2'] = industry['low'].shift(2)
    industry['low_t3'] = industry['low'].shift(3)
    industry['low_t4'] = industry['low'].shift(4)
    industry['low_t5'] = industry['low'].shift(5)
    industry['low_t6'] = industry['low'].shift(6)
    industry['low_t7'] = industry['low'].shift(7)
    industry['low_t8'] = industry['low'].shift(8)
    industry['low_t9'] = industry['low'].shift(9)
    industry['low_t10'] = industry['low'].shift(10)
    industry['amount_t1'] = industry['amount'].shift(1)
    industry['amount_t2'] = industry['amount'].shift(2)
    industry['amount_t3'] = industry['amount'].shift(3)
    industry['amount_t4'] = industry['amount'].shift(4)
    industry['amount_t5'] = industry['amount'].shift(5)
    industry['amount_t6'] = industry['amount'].shift(6)
    industry['amount_t7'] = industry['amount'].shift(7)
    industry['amount_t8'] = industry['amount'].shift(8)
    industry['amount_t9'] = industry['amount'].shift(9)
    industry['amount_t10'] = industry['amount'].shift(10)
    industry['SMA_10_rate_1'] = (industry['SMA_10'] - industry['SMA_10'].shift(1))
    industry['SMA_10_rate_2'] = (industry['SMA_10'].shift(1) - industry['SMA_10'].shift(2))
    industry['SMA_10_rate_3'] = (industry['SMA_10'].shift(2) - industry['SMA_10'].shift(3))
    industry['SMA_10_rate_4'] = (industry['SMA_10'].shift(3) - industry['SMA_10'].shift(4))
    industry['SMA_10_rate_5'] = (industry['SMA_10'].shift(4) - industry['SMA_10'].shift(5))
    industry['SMA_5_rate_1'] = (industry['SMA_5'] - industry['SMA_5'].shift(1))
    industry['SMA_5_rate_2'] = (industry['SMA_5'].shift(1) - industry['SMA_5'].shift(2))
    industry['SMA_5_rate_3'] = (industry['SMA_5'].shift(2) - industry['SMA_5'].shift(3))
    industry['SMA_5_rate_4'] = (industry['SMA_5'].shift(3) - industry['SMA_5'].shift(4))
    industry['SMA_5_rate_5'] = (industry['SMA_5'].shift(4) - industry['SMA_5'].shift(5))
    industry['SMA_15_rate_1'] = (industry['SMA_15'] - industry['SMA_15'].shift(1))
    industry['SMA_15_rate_2'] = (industry['SMA_15'].shift(1) - industry['SMA_15'].shift(2))
    industry['SMA_15_rate_3'] = (industry['SMA_15'].shift(2) - industry['SMA_15'].shift(3))
    industry['SMA_15_rate_4'] = (industry['SMA_15'].shift(3) - industry['SMA_15'].shift(4))
    industry['SMA_15_rate_5'] = (industry['SMA_15'].shift(4) - industry['SMA_15'].shift(5))
    industry['y'] = (industry['close'] - industry['close_t1'])/industry['close']
    industry['y'] = industry['y'].apply(lambda x: '1' if x>=0 else '0')
    industry['y'] = industry['y'].astype(int)
    industry['y'] = industry['y'].shift(-1)
    industry_now = industry.tail(1)
    industry.dropna(axis=0, inplace=True)
    drop_cols = ['date']
    industry = industry.drop(drop_cols, 1)
    valid_size = 0.25
    valid_split_idx = int(industry.shape[0] * (1 - (valid_size)))
    train_df = industry.loc[:valid_split_idx].copy()
    valid_df = industry.loc[valid_split_idx + 1:].copy()
    y_train = train_df['y'].copy()
    X_train = train_df.drop(['y'], 1)
    y_valid = valid_df['y'].copy()
    X_valid = valid_df.drop(['y'], 1)
    parameters = {
    'n_estimators': [3,4,5,6,7],
    'learning_rate': [0.02,0.01,0.007],
    'max_depth': [3,4,5,6,7],
    'gamma': [0.02,0.01,0.007],
    }
    model = xgb.XGBClassifier(use_label_encoder=False, eval_metric = 'logloss').fit(X_train, y_train)
    clf = GridSearchCV(model, parameters)
    clf.fit(X_train, y_train)
    model = xgb.XGBClassifier(**clf.best_params_)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    accuracy = accuracy_score(y_valid, y_pred)
    f1 = f1_score(y_valid, y_pred)
    now = industry_now.drop(['date', 'y'], 1).loc[:].copy()
    y_now = model.predict(now)[0]
    print(y_now)
    dic = {'industry': s, 'accuracy':accuracy, 'f1': f1, 'pred': y_now}
    industry_result.append(dic)

/Users/zhangjinghang/.conda/envs/ML/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:19:58] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
1.0


/Users/zhangjinghang/.conda/envs/ML/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:21:34] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.0


/Users/zhangjinghang/.conda/envs/ML/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[12:09:35] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.0


/Users/zhangjinghang/.conda/envs/ML/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:11:04] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.0


[12:12:34] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
1.0


/Users/zhangjinghang/.conda/envs/ML/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [67]:
industry_pred_result = pd.DataFrame(industry_result)
industry_pred_result['score'] = industry_pred_result['f1'] * industry_pred_result['pred']
industry_pred_result.sort_values('score', axis = 0, ascending=False, inplace=True)
industry_pred_result

,industry,accuracy,f1,pred,score
46,汽车零部件,0.553167,0.712309,1.0,0.712309
7,电力设备,0.546804,0.707011,1.0,0.707011
6,电力,0.537330,0.698600,1.0,0.698600
57,物流,0.533105,0.692250,1.0,0.692250
42,贸易,0.533105,0.690854,1.0,0.690854
...,...,...,...,...,...
47,汽车整车,0.507919,0.539683,0.0,0.000000
34,金属新材料,0.518265,0.672360,0.0,0.000000
44,农业服务,0.520548,0.574037,0.0,0.000000
35,教育,0.501359,0.627411,0.0,0.000000


In [68]:
industry_pred_result.to_excel("./ind_pred_result.xlsx")


In [4]:
import datetime
year = datetime.datetime.now().year
month = datetime.datetime.now().month
day = datetime.datetime.now().day
print('year: ', year)
print('month: ', month)
print('day: ', day)

time = str(year) + str('%02d'%month) + str('%02d'%day)
print('time: ', time)


year:  2021
month:  10
day:  4
time:  20211004


In [26]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.enableHiveSupport().getOrCreate()
# spark.sql('create database industry')
spark.sql("select * from industry.xgb_model_all20211015").show(100)


+--------------+-------------------+
|          name|              score|
+--------------+-------------------+
|    房地产开发| 0.3519918918609619|
|    房地产服务|0.34953439235687256|
|          中药|0.33928990364074707|
|    金属新材料|0.32972368597984314|
|    非金属材料| 0.3272510766983032|
|          物流|0.32714027166366577|
|      医药商业| 0.2930111587047577|
|      包装印刷|0.26012223958969116|
|    计算机应用|0.24737480282783508|
|    汽车零部件|0.22200104594230652|
|  化工合成材料| 0.6775765419006348|
|  煤炭开采加工| 0.6670255064964294|
|      通信服务| 0.6531074643135071|
|  食品加工制造| 0.6403070688247681|
|    景点及旅游| 0.6399438381195068|
|    光学光电子|  0.637273371219635|
|      化学制药| 0.4023072123527527|
|      消费电子| 0.3912521004676819|
|    计算机设备|0.38351884484291077|
|      生物制品| 0.3789963126182556|
|    非汽车交运| 0.3551666736602783|
|    自动化设备| 0.3521571755409241|
|      通用设备|0.46636390686035156|
|      黑色家电| 0.4622817933559418|
|  种植业与林业|0.45296669006347656|
|  石油加工贸易|0.45177072286605835|
|      服装家纺|0.42875510454177856|
|      纺织制造| 0.

In [23]:
spark.sql('use industry')

DataFrame[]

In [24]:
spark.sql('show tables').show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
|industry|               test1|      false|
|industry|xgb_model_all2021...|      false|
|industry|xgb_model_all2021...|      false|
|industry|xgb_model_all2021...|      false|
|industry|xgb_model_all2021...|      false|
|industry|xgb_model_all2021...|      false|
+--------+--------------------+-----------+



In [20]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.enableHiveSupport().getOrCreate()

In [3]:
spark.sql("create database industry")

DataFrame[]

In [28]:
df = spark.sql("select * from industry.test1")

In [29]:
df.write.saveAsTable("industry.test2")